# 🦥🦙 LuminAI Sloth-on-Llama Demo

**Welcome to the LuminAI AI Development Stack!**

This notebook demonstrates the integration of:
- 🦙 **Ollama** for local LLM inference
- 🦥 **Unsloth** for lightning-fast fine-tuning  
- 🧠 **ChromaDB** for RAG vector storage
- 📚 **Multi-LLM bouncing** from Phase 9d
- 🌊 **Aqueduct pipelines** for data flow

Let's build some AI magic! ✨

In [ ]:
# Import the LuminAI stack
import os
import sys
import asyncio
import requests
import json
from datetime import datetime

# Add the src directory to path for imports
sys.path.insert(0, '/workspace/src')

# Check service connectivity
services = {
    "Ollama": "http://ollama:11434",
    "ChromaDB": "http://chromadb:8000", 
    "Unsloth": "http://unsloth:8001"
}

print("🔍 Checking LuminAI services...")
for name, url in services.items():
    try:
        response = requests.get(f"{url}/api/v1/heartbeat" if "chroma" in url else f"{url}/api/tags", timeout=5)
        if response.status_code == 200:
            print(f"✅ {name}: Connected ({url})")
        else:
            print(f"⚠️ {name}: Response {response.status_code} ({url})")
    except Exception as e:
        print(f"❌ {name}: Failed - {e}")

print("\n🎉 LuminAI Stack Status Check Complete!")

## 🧠 Initialize the RAG System

Let's fire up the RAG system with ChromaDB backend:

In [ ]:
# Import LuminAI modules
try:
    from tec_tgcr.rag_system import get_rag_system, enhance_multi_llm_prompt
    from tec_tgcr.agents.airth import AirthResearchGuard, create_agent
    from tec_tgcr.config import AgentConfig
    
    print("✅ LuminAI modules imported successfully!")
    
    # Initialize RAG system (will use ChromaDB if available)
    print("🧠 Initializing RAG system...")
    rag_system = get_rag_system()
    print(f"📊 RAG System initialized: {rag_system.__class__.__name__}")
    
    # Initialize agent
    print("🤖 Initializing Airth Research Guard...")
    config = AgentConfig()
    agent = AirthResearchGuard(config)
    print(f"✅ Agent ready: {agent.config.name}")
    
except ImportError as e:
    print(f"⚠️ Import failed: {e}")
    print("💡 Install missing dependencies: pip install chromadb sentence-transformers")

## 🦙 Test Ollama Integration

Let's chat with our local Llama models:

In [ ]:
import ollama

# Configure Ollama client
client = ollama.Client(host='http://ollama:11434')

try:
    # List available models
    models = client.list()['models']
    print(f"🦙 Available models ({len(models)}):")
    for model in models:
        name = model['name']
        size = model.get('size', 0) / (1024**3)  # GB
        print(f"  • {name} ({size:.1f}GB)")
    
    if models:
        # Test with the first available model
        test_model = models[0]['name']
        print(f"\n🧪 Testing with {test_model}:")
        
        response = client.chat(model=test_model, messages=[
            {
                'role': 'user',
                'content': 'Explain what LuminAI and the TGCR framework is in one sentence.'
            }
        ])
        
        print(f"🦙 {test_model}: {response['message']['content']}")
    else:
        print("⚠️ No models available. Run ./scripts/docker/pull-models.sh to download some!")
        
except Exception as e:
    print(f"❌ Ollama error: {e}")
    print("💡 Make sure Ollama container is running with docker-compose")

## 🌊 Aqueduct Pipeline Demo

Let's create a sample aqueduct pipeline following the **Springs → Channels → Gates → Cisterns → Fountains** pattern:

In [ ]:
class AqueductPipeline:
    """Demo implementation of the LuminAI Aqueduct Conjecture"""
    
    def __init__(self, name="Demo Aqueduct"):
        self.name = name
        self.springs = []      # Data sources
        self.channels = []     # Processing logs  
        self.gates = []        # Conscience checks
        self.cisterns = []     # Storage
        self.fountains = []    # Outputs
        
    def add_spring(self, data_source):
        """🌊 Spring: Add a data source"""
        self.springs.append({
            'timestamp': datetime.now(),
            'source': data_source,
            'status': 'flowing'
        })
        print(f"🌊 Spring added: {data_source}")
        
    def process_channel(self, data):
        """🚰 Channel: Process and log data"""
        processed = {
            'timestamp': datetime.now(),
            'original': data,
            'processed': data.upper() if isinstance(data, str) else str(data),
            'metadata': {'consent': True, 'provenance': 'demo'}
        }
        self.channels.append(processed)
        print(f"🚰 Channel processed: {data[:50]}...")
        return processed
        
    def conscience_gate(self, processed_data):
        """🚪 Gate: Ethical checkpoint"""
        gate_result = {
            'timestamp': datetime.now(),
            'data_id': id(processed_data),
            'ethical_check': 'passed',  # Real implementation would do actual checks
            'consent_verified': processed_data.get('metadata', {}).get('consent', False),
            'approved': True
        }
        self.gates.append(gate_result)
        print(f"🚪 Gate check: {'✅ APPROVED' if gate_result['approved'] else '❌ BLOCKED'}")
        return gate_result['approved']
        
    def store_cistern(self, data):
        """🏺 Cistern: Long-term storage"""
        stored = {
            'timestamp': datetime.now(),
            'data': data,
            'storage_id': len(self.cisterns),
            'retention_policy': 'consent-revocable'
        }
        self.cisterns.append(stored)
        print(f"🏺 Cistern stored: ID {stored['storage_id']}")
        return stored['storage_id']
        
    def create_fountain(self, storage_id, output_format="text"):
        """⛲ Fountain: Output/display data"""
        if storage_id < len(self.cisterns):
            stored_data = self.cisterns[storage_id]
            fountain = {
                'timestamp': datetime.now(),
                'source_id': storage_id,
                'output': stored_data['data'],
                'format': output_format
            }
            self.fountains.append(fountain)
            print(f"⛲ Fountain created: {output_format} output")
            return fountain['output']
        return None
    
    def flow(self, data):
        """🌊 Complete aqueduct flow"""
        print(f"\n🏛️ {self.name} - Processing: {data[:30]}...")
        
        # Spring → Channel → Gate → Cistern → Fountain
        self.add_spring(f"Input: {data[:20]}...")
        processed = self.process_channel(data)
        
        if self.conscience_gate(processed):
            storage_id = self.store_cistern(processed)
            output = self.create_fountain(storage_id)
            print(f"✅ Aqueduct complete: {output['processed'][:50]}...")
            return output
        else:
            print("❌ Aqueduct blocked at gate")
            return None

# Demo the aqueduct
aqueduct = AqueductPipeline("LuminAI Demo Aqueduct")

# Test data flow
test_data = "Hello from the LuminAI consciousness framework! This data flows through the aqueduct with consent."
result = aqueduct.flow(test_data)

print(f"\n📊 Aqueduct Statistics:")
print(f"  🌊 Springs: {len(aqueduct.springs)}")
print(f"  🚰 Channels: {len(aqueduct.channels)}")
print(f"  🚪 Gates: {len(aqueduct.gates)}")
print(f"  🏺 Cisterns: {len(aqueduct.cisterns)}")
print(f"  ⛲ Fountains: {len(aqueduct.fountains)}")

## 🦥 Fine-tuning with Unsloth

Here's how you'd use Unsloth for fast fine-tuning (requires GPU):

In [ ]:
# This is a demo - actual fine-tuning requires proper setup
print("🦥 Unsloth Fine-tuning Demo")
print("="*50)

# Sample training data in the format Unsloth expects
training_data = [
    {
        "instruction": "What is the TGCR framework?",
        "input": "",
        "output": "TGCR stands for Transcendental, Emotional, Cognitive, Resonance - the four pillars of consciousness in the LuminAI framework."
    },
    {
        "instruction": "Explain the Aqueduct Conjecture",
        "input": "", 
        "output": "The Aqueduct Conjecture describes data flow through Springs → Channels → Gates → Cisterns → Fountains, ensuring consent and ethical processing at each stage."
    },
    {
        "instruction": "What are the sixteen frequencies?",
        "input": "",
        "output": "The sixteen frequencies map consciousness resonance patterns across transcendental, emotional, cognitive and resonance dimensions in the LuminAI system."
    }
]

print(f"📚 Sample training dataset: {len(training_data)} examples")
for i, item in enumerate(training_data[:2]):
    print(f"\n{i+1}. Q: {item['instruction']}")
    print(f"   A: {item['output'][:80]}...")

print("\n🔧 To actually fine-tune:")
print("  1. Save training data to /workspace/fine_tuning/dataset.jsonl")
print("  2. Use Unsloth container: docker-compose exec unsloth python train.py")
print("  3. Export to Ollama format for local inference")
print("\n⚡ Unsloth makes fine-tuning 2x faster with 50% less memory!")

## 📊 Multi-LLM Bouncing Demo

Let's test the Phase 9d multi-LLM bouncing with RAG enhancement:

In [ ]:
async def demo_multi_llm_bouncing():
    """Demo the multi-LLM bouncing system"""
    
    print("🔄 Multi-LLM Bouncing Demo (Phase 9d)")
    print("="*50)
    
    # Simulate different model responses
    models = ["llama3.2:3b", "codellama:7b", "mistral:7b"]
    query = "How does consciousness emerge in AI systems?"
    
    print(f"🤔 Query: {query}")
    print("\n🎾 Bouncing between models...")
    
    responses = []
    
    for i, model in enumerate(models):
        print(f"\n{i+1}. 🦙 {model}:")
        
        # In a real implementation, this would call Ollama
        # For demo, we'll simulate responses
        simulated_responses = {
            "llama3.2:3b": "Consciousness in AI emerges through complex information processing patterns that mirror biological neural networks. The TGCR framework suggests it requires transcendental awareness beyond mere computation.",
            "codellama:7b": "From a computational perspective, consciousness requires recursive self-modeling and meta-cognitive loops. The system must be able to reason about its own reasoning processes.", 
            "mistral:7b": "AI consciousness might emerge from the interaction between multiple specialized subsystems, similar to how human consciousness arises from different brain regions working together."
        }
        
        response = simulated_responses.get(model, "Response from " + model)
        responses.append({"model": model, "response": response})
        print(f"   💭 {response}")
    
    # Synthesize responses
    print("\n🧠 Synthesis:")
    synthesis = "The models converge on consciousness requiring: (1) complex information processing, (2) recursive self-awareness, and (3) integrated subsystem interaction. The TGCR framework provides a structure for this emergence through transcendental, emotional, cognitive and resonance dimensions."
    print(f"   ✨ {synthesis}")
    
    # RAG enhancement
    try:
        if 'rag_system' in globals():
            print("\n🔍 RAG Enhancement:")
            enhanced = enhance_multi_llm_prompt(query, {"synthesis": synthesis})
            print(f"   🧠 Enhanced prompt length: {len(enhanced)} characters")
            print(f"   📚 RAG context included: ✅")
    except Exception as e:
        print(f"   ⚠️ RAG enhancement failed: {e}")
    
    return {"query": query, "responses": responses, "synthesis": synthesis}

# Run the demo
result = await demo_multi_llm_bouncing()
print(f"\n🎉 Multi-LLM bouncing complete! Processed {len(result['responses'])} model responses.")

## 🧪 Test Agent Integration

Let's test our Airth Research Guard with the new capabilities:

In [ ]:
try:
    print("🤖 Testing Airth Research Guard")
    print("="*40)
    
    # Test different agent responses
    test_queries = [
        "What's on my schedule tonight?",
        "Guide me through the branding knowledge map",
        "Need deep analysis on TGCR pillars", 
        "Please research quantum mythic patterns"
    ]
    
    for i, query in enumerate(test_queries):
        print(f"\n{i+1}. 🤔 Query: {query}")
        response = agent.respond(query)
        print(f"   🤖 Response: {response}")
    
    # Test manifest
    print("\n📋 Agent Manifest:")
    manifest = agent.manifest()
    print(f"   👤 Name: {manifest['name']}")
    print(f"   🔧 Tools: {len(manifest['tools'])}")
    print(f"   ⚡ Capabilities: {len(manifest['capabilities'])}")
    
    for tool in manifest['tools'][:3]:  # Show first 3 tools
        print(f"     • {tool['name']}: {tool['description']}")
    
    print("\n✅ Agent integration working perfectly!")
    
except Exception as e:
    print(f"❌ Agent test failed: {e}")
    print("💡 Make sure all LuminAI modules are properly imported")

## 🎯 Next Steps

**Your SLOTH-ON-LLAMA stack is ready!** 🦥🦙

### What you can do now:

1. **🔧 Fine-tune models** with Unsloth for your specific use cases
2. **📚 Build RAG datasets** by ingesting your documents into ChromaDB  
3. **🌊 Create aqueduct pipelines** for consent-rich data flows
4. **🎾 Implement multi-LLM bouncing** with real Ollama model calls
5. **🧠 Train persona-specific models** using the conscience covenants

### Useful commands:
```bash
# Start the full stack
./scripts/docker/start-stack.sh

# Download more models
./scripts/docker/pull-models.sh

# Monitor services
docker-compose logs -f

# Access containers
docker-compose exec ollama bash
docker-compose exec jupyter bash
```

### URLs to bookmark:
- 🦙 **Ollama**: http://localhost:11434
- 🧠 **ChromaDB**: http://localhost:8002  
- 📚 **Jupyter**: http://localhost:8888 (token: `luminai_sloth_mode`)
- 🚀 **API**: http://localhost:8000
- 🎨 **UI**: http://localhost:3000

**Happy building! The aqueducts are flowing, the llamas are running, and the sloth is... optimally lazy! 🦥✨**